## Computer Vision CS766
Final Project
2023

Hailey Johnson

### Purpose 
I am a researcher in Human-Computer Interaction (HCI) and I took this class to assist me in my research project. I am working to develop assistive technologies for adults with Down syndrome after they have left school with the aim to increase job oppurtunities. For the initial evaluation of the technologies, we need to test if a user can use the system to complete a multi-step task. Our plan is to walk our participants through a Lego building task. 

For this project I have attempted to construct an object detection system that informs the user if they have correctly completed the current step or if they need to try again. 

### Process
First, I took images of indiviudal legos from my lego set to create the basis of my dataset. (Note: they are all in the same lighting conditions) 

Second, I took images of each step in the multi-step bulding process along with wrong step images for testing purposes.

Third, I created a looped process that checks each update the user makes in their building process to determine if they have completed the correct step. If they have not, it rechecks that same step until they have completed it correctly. 

I performed image processing techniques learned in class, isolation of the new Lego that was placed in the construction, detection to determine if the new Lego is the correct peice, then detection if the peice is placed correctly on the whole construction. 

### Correct step videos:
1. base.mp4
2. step1.mp4
3. step2.mp4
4. step3.mp4
5. step4.mp4
6. step5.mp4
7. step6.mp4
8. step7.mp4
9. step8.mp4
10. step9.mp4
11. step10.mp4
12. step11.mp4
13. step12.mp4

### Lego videos:
1. lego1.mp4
2. lego2.mp4
3. lego3.mp4
4. lego4.mp4
5. lego5.mp4
6. lego6.mp4
7. lego7.mp4
8. lego8.mp4
9. lego9.mp4

### Testing videos:
1. wrong2.mp4
2. wrong4.mp4
3. wrong5.mp4
4. wrong6.mp4
5. wrong8.mp4
6. wrongEnd.mp4

In [ ]:
import cv2
import numpy as np

videos = [
    'step1.mp4',
    'step2.mp4',
    'step3.mp4'
]

imgs = [
    'step1_0003.png',
    'step2_0003.png',
    'step3_0003.png'
]

colors = [
    [[228, 101, 227], [106, 51, 132]],
    [[255, 169, 231], [125, 38, 148]],
    [[224,  83,  90], [112,  33,  39]]
]

In [ ]:
#find upper color

def click_callback(event, x, y, flags, param):
    img, pixel_color, click_counter = param  # Unpack the parameters
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel_color[0] = img[y, x]
        print("Color at ({}, {}): {}".format(x, y, pixel_color[0]))
        click_counter[0] += 1

def find_color(path):
    img = cv2.imread(path)
    
    # Preprocess the img
    #glare filtering
    # Convert the frame from RGB to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Define the thresholds for the glare color
    lower_glare = np.array([0, 0, 200])
    upper_glare = np.array([180, 150, 255])
    # Create a mask for the glare color
    mask = cv2.inRange(hsv, lower_glare, upper_glare)
    # Perform morphological operations to remove noise and fill in gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.erode(mask, kernel, iterations=2)
    mask = cv2.dilate(mask, kernel, iterations=2)
    # Apply the matte effect to reduce the intensity of the glare
    img[mask > 0] -= 40
    
    cv2.namedWindow('image')
    click_counter = [0]
    pixel_color = [None]  # Define the pixel_color as a mutable list
    cv2.setMouseCallback('image', click_callback, param=(img, pixel_color, click_counter))

    while True:
        cv2.imshow('image', img)
        key = cv2.waitKey(1) & 0xFF
        if click_counter[0] >= 1 or key == ord('q'):
            break
    print("Color : {}".format(pixel_color[0]))
    cv2.destroyAllWindows()
    
    return pixel_color

In [ ]:


for idx in range(len(videos)):
    # Load the video and image files
    video_path = videos[idx]
    cap = cv2.VideoCapture(video_path)

    output_path = "test_output_" + video_path
    # Get the video dimensions and FPS
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Create the video writer object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    #find color range: upper limit first, lower limit second
    pixel_color_upper = find_color(imgs[idx])
    pixel_color_rgb_upper = cv2.cvtColor(np.array(pixel_color_upper), cv2.COLOR_BGR2RGB)
    pixel_color_lower = find_color(imgs[idx])
    pixel_color_rgb_lower = cv2.cvtColor(np.array(pixel_color_lower), cv2.COLOR_BGR2RGB)
    
    lower_range = np.array(pixel_color_rgb_lower)
    upper_range = np.array(pixel_color_rgb_upper)

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

       # Preprocess the frame
        #glare filtering
        # Convert the frame from RGB to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # Define the thresholds for the glare color
        lower_glare = np.array([0, 0, 200])
        upper_glare = np.array([180, 150, 255])
        # Create a mask for the glare color
        mask = cv2.inRange(hsv, lower_glare, upper_glare)
        # Perform morphological operations to remove noise and fill in gaps
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.erode(mask, kernel, iterations=2)
        mask = cv2.dilate(mask, kernel, iterations=2)
        # Apply the matte effect to reduce the intensity of the glare
        frame[mask > 0] -= 40

        # Perform color detection
        mask = cv2.inRange(hsv, lower_range, upper_range)

        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 300:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break


        # write the frame to the output video
        out.write(frame)

        # Display the result
        cv2.imshow('result', frame)
        if cv2.waitKey(1) == ord('q'):
            break

    # Display the result
    out.release()
    cap.release()
    cv2.destroyAllWindows()